In [ ]:
# !pip install stravalib
# !pip install seaborn
# !pip install folium

In [ ]:
import time
import pickle
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import folium
import pytz
import datetime as dt
import ast
import plotly.graph_objects as go
import os
import random
import glob

from stravalib import Client
pd.set_option("display.max_columns", 100)

### Load Existing Data

In [ ]:
df_heartrate = pd.read_csv( '../user_data/strava_heartrate.csv')
df_activity = pd.read_csv( '../user_data/strava_activities.csv')

## Get song data

In [ ]:
def only_get_songs_between_activity_and_end_ts(df_activity, df_song, activity_id=None):
    ''' 
    Gets songs only for a given activity, defined by activity_id
    
    Parameters
    ----------
    song_df : pd.DataFrame
        DataFrame containing song data
    activity_df : pd.DataFrame
        DataFrame containing activity data
    activity_id : int
        ID of activity to filter songs for
    
    Returns
    -------
    pd.DataFrame
        DataFrame containing songs only for the given activity
    '''

    if activity_id is not None:
        df_activity = df_activity[df_activity['id'] == activity_id]
    
    min_activity_start_time = df_activity['start_date_local'].min()
    max_activity_end_time = df_activity['end_date'].max()
    
    #convert df_song['song_start_ts_utc'] to est
    df_song['song_start_ts_local'] = df_song['song_start_ts_utc'].apply(lambda x: convert_utc_to_est(x))

    return df_song[
        (df_song['song_start_ts_local'].between(min_activity_start_time, max_activity_end_time)) &
        (df_song['song_end_ts'].between(min_activity_start_time, max_activity_end_time))
    ]


def songs_cumsum_time(df_song):
    '''
    Function to calculate the cumulative time of songs
    '''

    # Convert to Minutes  and round
    df_song['song_length_min']  = df_song['song_length'].dt.total_seconds()/60
    df_song['song_length_min'] = df_song['song_length_min'].round(5)      

    # seconds
    df_song['song_length_sec']  = df_song['song_length'].dt.total_seconds()

    # cumulative time on an activity
    df_song['song_length_sec_cum']= df_song['song_length_sec'].cumsum()
    return


# Function to convert UTC to EST
def convert_utc_to_est(utc_time):
    '''
    Function to convert UTC to EST
    '''
    # utc_time = utc_time.replace(tzinfo=pytz.utc)  # Ensure UTC timezone
    est_time = utc_time.astimezone(pytz.timezone('US/Eastern'))  # Convert to EST
    return est_time


In [ ]:
df_songs_import = pd.read_csv("/Users/dandeangelis/projects/strava-spotify-tracking/user_data/recently_played.csv")

In [ ]:
df_songs_import

In [ ]:
df_songs = df_songs_import
# Calculate Song_End Timestamp
df_songs['song_start_ts_utc'] = pd.to_datetime(df_songs['song_start_ts_utc'])

df_songs['duration_ms'] = df_songs['duration_ms']/1000
df_songs['song_end_ts'] = df_songs['song_start_ts_utc'] + pd.to_timedelta(df_songs['duration_ms'], unit='ms')
df_songs['song_length'] = df_songs['song_end_ts'] - df_songs['song_start_ts_utc']  

# Convert to Minutes  and round
df_songs['song_length_min']  = df_songs['song_length'].dt.total_seconds()/60
df_songs['song_length_min'] = df_songs['song_length_min'].round(5)      

# seconds
df_songs['song_length_sec']  = df_songs['song_length'].dt.total_seconds()

In [ ]:
df_single_activity = df_activity[df_activity.id == 10634057895]
df_songs_by_activity = only_get_songs_between_activity_and_end_ts(df_single_activity, df_songs, activity_id=10634057895)
df_songs_by_activity

#10540604927 // 10634057895

In [ ]:
df_heartrate.dtypes

In [ ]:
df_merged_activity = pd.merge(df_single_activity,df_heartrate, on=['id'])
df_merged_activity

In [ ]:
print(type(df_merged_activity[df_merged_activity.id == 10634057895 ].time))

In [ ]:
df_merged_activity[df_merged_activity['id']==10634057895]

In [ ]:
# Cumulative time of songs running
df_songs_by_activity['song_length_sec_cum']= df_songs_by_activity['song_length_sec'].cumsum()



In [ ]:
df_activity_final = df_merged_activity[df_merged_activity['id']==10634057895]

# Zip 'time' and 'heartrate' columns together into a new column 'time_heartrate'
df_activity_final['time_heartrate'] = list(zip(df_activity_final['time'], df_activity_final['heartrate']))

df_activity_final


In [17]:
import ast
# Ensure df_single_activity and df_song_filtered are not empty
if not df_songs_by_activity.empty and not df_activity_final.empty:
    # Create figure
    fig = go.Figure()

# Heart rate line
    for _, row in df_activity_final.iterrows():
        fig.add_trace(go.Scatter(
            x=ast.literal_eval(row['time_heartrate'][0]),
            y=ast.literal_eval(row['time_heartrate'][1]),
            mode='lines+markers',
            name='Heart Rate'
        ))


    # Shaded song segments with hover text and annotations
    for i, row in df_songs_by_activity.iterrows():
        fillcolor = f"rgba({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)}, 0.2)"

        fig.add_vrect(
            x0=row['song_length_sec_cum'],
            x1=df_songs_by_activity['song_length_sec_cum'] if i < len(df_songs_by_activity) - 1 else None,
            fillcolor=fillcolor,
            layer="below",
            name=str(df_songs_by_activity['track'])
        )

        # Annotate track title
        fig.add_annotation(
            x=row['song_length_sec_cum'],
            y=0,
            text=row['track'],
            showarrow=False,
            font=dict(size=8, color='rgba(0, 0, 0, 1)'),
            xanchor='left',
            yanchor='bottom',
            textangle=90
        )

    # Customize layout
    fig.update_layout(
        xaxis_title="Time (seconds)",
        yaxis_title="Heart Rate (bpm)",
        title="Runner's Heart Rate with Music Segments",
        hovermode="x"
    )

    # Display interactive plot
    fig.show()

else:
    print("DataFrame df_song_filtered or df_single_activity is empty. Please check your data.")
